In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
import re
import requests
import json


In [12]:
vespa_df = pd.read_csv("GB_vespa_df.csv")
vespa_df = vespa_df.rename(columns={'mutation_pos':'Protein_position'})
vespa_df

,Uniprot_ID,SYMBOL,VESPAl,mutation_from,Protein_position,mutation_to
0,P0DMS8,ADORA3,0.444900,S,317,C
1,P0DMS8,ADORA3,0.431531,N,316,Y
2,P0DMS8,ADORA3,0.394209,E,314,A
3,P0DMS8,ADORA3,0.416500,T,311,I
4,P0DMS8,ADORA3,0.438174,D,310,V
...,...,...,...,...,...,...
15486,Q9H244,P2RY12,0.340720,D,5,G
15487,Q9H244,P2RY12,0.340720,D,5,H
15488,Q9H244,P2RY12,0.265001,D,5,N
15489,Q9H244,P2RY12,0.181930,V,4,I


In [13]:
icd10_df = pd.read_csv('VEPwithGB_icd10_0323.csv')
icd10_df

/Users/joannahench/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (8,9,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Location,Allele,Protein_position,SYMBOL,Amino_acids,loc_m,SIFT,PolyPhen,BayesDel_addAF_score,BayesDel_noAF_score,...,coding_description,category,AC,AF,BETA,SE,AF.Cases,AF.Controls,Pvalue,vcf
0,11:113412715-113412715,C,298,DRD2,K/E,113412715,tolerated(0.34),benign(0.012),-0.260005,-0.140988,...,NaN,Chapter II Neoplasms | C15-C26 Malignant neopl...,1907,0.002415,0.431260,0.53696,0.003472,0.002413,0.421890,11 113412715 . T C
1,11:113412715-113412715,C,298,DRD2,K/E,113412715,tolerated(0.34),benign(0.012),-0.260005,-0.140988,...,NaN,Chapter II Neoplasms | C15-C26 Malignant neopl...,1907,0.002415,-0.246980,0.61625,0.001845,0.002416,0.688580,11 113412715 . T C
2,11:113412715-113412715,C,298,DRD2,K/E,113412715,tolerated(0.34),benign(0.012),-0.260005,-0.140988,...,NaN,Chapter II Neoplasms | C15-C26 Malignant neopl...,1907,0.002415,1.305800,1.07300,0.005556,0.002413,0.223580,11 113412715 . T C
3,11:113412715-113412715,C,298,DRD2,K/E,113412715,tolerated(0.34),benign(0.012),-0.260005,-0.140988,...,NaN,Chapter II Neoplasms | C15-C26 Malignant neopl...,1907,0.002415,-0.141640,0.27161,0.002068,0.002417,0.602020,11 113412715 . T C
4,11:113412715-113412715,C,298,DRD2,K/E,113412715,tolerated(0.34),benign(0.012),-0.260005,-0.140988,...,NaN,Chapter II Neoplasms | C15-C26 Malignant neopl...,1907,0.002415,0.071904,0.52037,0.002584,0.002415,0.890100,11 113412715 . T C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
610533,X:114907411-114907411,C,458,HTR2C,V/A,114907411,deleterious(0),benign(0.309),-0.00911951,-0.250876,...,NaN,Chapter XXI Factors influencing health status ...,3,0.000004,-1.007700,7.82990,0.000000,0.000004,0.897600,X 114907411 . T C
610534,X:114907411-114907411,C,458,HTR2C,V/A,114907411,deleterious(0),benign(0.309),-0.00911951,-0.250876,...,NaN,Chapter XXI Factors influencing health status ...,3,0.000004,5.129800,2.33040,0.000049,0.000003,0.027721,X 114907411 . T C
610535,X:114907411-114907411,C,458,HTR2C,V/A,114907411,deleterious(0),benign(0.309),-0.00911951,-0.250876,...,NaN,Chapter XXI Factors influencing health status ...,3,0.000004,-1.015900,4.91580,0.000000,0.000004,0.836280,X 114907411 . T C
610536,X:114907411-114907411,C,458,HTR2C,V/A,114907411,deleterious(0),benign(0.309),-0.00911951,-0.250876,...,NaN,Chapter XXI Factors influencing health status ...,3,0.000004,-1.022800,3.18950,0.000000,0.000004,0.748460,X 114907411 . T C


In [14]:
icd10_df['mutation_from'] = icd10_df['Amino_acids'].str.split('/',expand=True)[0]
icd10_df['mutation_to'] = icd10_df['Amino_acids'].str.split('/',expand=True)[1]


icd10_df

,Location,Allele,Protein_position,SYMBOL,Amino_acids,loc_m,SIFT,PolyPhen,BayesDel_addAF_score,BayesDel_noAF_score,...,AC,AF,BETA,SE,AF.Cases,AF.Controls,Pvalue,vcf,mutation_from,mutation_to
0,11:113412715-113412715,C,298,DRD2,K/E,113412715,tolerated(0.34),benign(0.012),-0.260005,-0.140988,...,1907,0.002415,0.431260,0.53696,0.003472,0.002413,0.421890,11 113412715 . T C,K,E
1,11:113412715-113412715,C,298,DRD2,K/E,113412715,tolerated(0.34),benign(0.012),-0.260005,-0.140988,...,1907,0.002415,-0.246980,0.61625,0.001845,0.002416,0.688580,11 113412715 . T C,K,E
2,11:113412715-113412715,C,298,DRD2,K/E,113412715,tolerated(0.34),benign(0.012),-0.260005,-0.140988,...,1907,0.002415,1.305800,1.07300,0.005556,0.002413,0.223580,11 113412715 . T C,K,E
3,11:113412715-113412715,C,298,DRD2,K/E,113412715,tolerated(0.34),benign(0.012),-0.260005,-0.140988,...,1907,0.002415,-0.141640,0.27161,0.002068,0.002417,0.602020,11 113412715 . T C,K,E
4,11:113412715-113412715,C,298,DRD2,K/E,113412715,tolerated(0.34),benign(0.012),-0.260005,-0.140988,...,1907,0.002415,0.071904,0.52037,0.002584,0.002415,0.890100,11 113412715 . T C,K,E
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
610533,X:114907411-114907411,C,458,HTR2C,V/A,114907411,deleterious(0),benign(0.309),-0.00911951,-0.250876,...,3,0.000004,-1.007700,7.82990,0.000000,0.000004,0.897600,X 114907411 . T C,V,A
610534,X:114907411-114907411,C,458,HTR2C,V/A,114907411,deleterious(0),benign(0.309),-0.00911951,-0.250876,...,3,0.000004,5.129800,2.33040,0.000049,0.000003,0.027721,X 114907411 . T C,V,A
610535,X:114907411-114907411,C,458,HTR2C,V/A,114907411,deleterious(0),benign(0.309),-0.00911951,-0.250876,...,3,0.000004,-1.015900,4.91580,0.000000,0.000004,0.836280,X 114907411 . T C,V,A
610536,X:114907411-114907411,C,458,HTR2C,V/A,114907411,deleterious(0),benign(0.309),-0.00911951,-0.250876,...,3,0.000004,-1.022800,3.18950,0.000000,0.000004,0.748460,X 114907411 . T C,V,A


In [15]:
# merge two dataframes
gb_with_vespa  = pd.merge(icd10_df,vespa_df,on=['Protein_position','mutation_from','mutation_to','SYMBOL'])
gb_with_vespa

,Location,Allele,Protein_position,SYMBOL,Amino_acids,loc_m,SIFT,PolyPhen,BayesDel_addAF_score,BayesDel_noAF_score,...,BETA,SE,AF.Cases,AF.Controls,Pvalue,vcf,mutation_from,mutation_to,Uniprot_ID,VESPAl
0,11:113416935-113416935,T,154,DRD2,V/I,113416935,tolerated(0.15),possibly_damaging(0.656),0.134914,0.141922,...,-1.00480,6.3053,0.000000,0.000015,0.873390,11 113416935 . C T,V,I,P14416,0.118742
1,11:113416935-113416935,T,154,DRD2,V/I,113416935,tolerated(0.15),possibly_damaging(0.656),0.134914,0.141922,...,-1.00340,7.2464,0.000000,0.000015,0.889870,11 113416935 . C T,V,I,P14416,0.118742
2,11:113416935-113416935,T,154,DRD2,V/I,113416935,tolerated(0.15),possibly_damaging(0.656),0.134914,0.141922,...,-0.99774,13.0160,0.000000,0.000015,0.938900,11 113416935 . C T,V,I,P14416,0.118742
3,11:113416935-113416935,T,154,DRD2,V/I,113416935,tolerated(0.15),possibly_damaging(0.656),0.134914,0.141922,...,-1.01020,3.2737,0.000000,0.000015,0.757640,11 113416935 . C T,V,I,P14416,0.118742
4,11:113416935-113416935,T,154,DRD2,V/I,113416935,tolerated(0.15),possibly_damaging(0.656),0.134914,0.141922,...,-1.00350,6.4900,0.000000,0.000015,0.877120,11 113416935 . C T,V,I,P14416,0.118742
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
555366,X:114907411-114907411,C,458,HTR2C,V/A,114907411,deleterious(0),benign(0.309),-0.00911951,-0.250876,...,-1.00770,7.8299,0.000000,0.000004,0.897600,X 114907411 . T C,V,A,P28335,0.463933
555367,X:114907411-114907411,C,458,HTR2C,V/A,114907411,deleterious(0),benign(0.309),-0.00911951,-0.250876,...,5.12980,2.3304,0.000049,0.000003,0.027721,X 114907411 . T C,V,A,P28335,0.463933
555368,X:114907411-114907411,C,458,HTR2C,V/A,114907411,deleterious(0),benign(0.309),-0.00911951,-0.250876,...,-1.01590,4.9158,0.000000,0.000004,0.836280,X 114907411 . T C,V,A,P28335,0.463933
555369,X:114907411-114907411,C,458,HTR2C,V/A,114907411,deleterious(0),benign(0.309),-0.00911951,-0.250876,...,-1.02280,3.1895,0.000000,0.000004,0.748460,X 114907411 . T C,V,A,P28335,0.463933


In [16]:
gb_with_vespa.to_csv('Genebass_with_VESPAl_df.csv',index=False)

## [not used]classification performance(seperately)

In [14]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report, roc_curve, auc, confusion_matrix

In [19]:
gpcr_vespa['VESPAl']

,Unnamed: 0,reference,protein,mutation_pos,mutation_from,mutation_to,ligand_name,ligand_idtype,ligand_id,ligand_class,...,exp_mu_effect_qual,exp_mu_effect_ligand_prop,exp_mu_ligand_ref,opt_receptor_expression,opt_basal_activity,opt_gain_of_activity,opt_ligand_emax,opt_agonist,Uniprot_ID,VESPAl
0,0,8903934,hrh1_human,194,T,A,(S)-cetirizine,ChEMBL Compound ID,CHEMBL1334217,Binding - unknown pharmacological activity,...,NaN,NaN,[3H]-mepyramine (radioligand),0.0,0.0,NaN,0.0,NaN,P35367,0.450196
1,8,8198587,hrh1_human,194,T,A,2-methylhistamine,ChEMBL Compound ID,CHEMBL12620,Binding - unknown pharmacological activity,...,NaN,NaN,[3H]-mepyramine (radioligand),126.0,0.0,NaN,0.0,NaN,P35367,0.450196
2,12,11809864,hrh1_human,194,T,A,(S)-cetirizine,ChEMBL Compound ID,CHEMBL1334217,Binding - unknown pharmacological activity,...,NaN,NaN,[3H]-mepyramine (radioligand),24.4,0.0,NaN,0.0,NaN,P35367,0.450196
3,19,8093027,hrh1_human,194,T,A,[3H]-mepyramine,ChEMBL Compound ID,CHEMBL511,Binding - unknown pharmacological activity,...,NaN,NaN,[3H]-mepyramine (radioligand),122.7,0.0,NaN,0.0,NaN,P35367,0.450196
4,20,8198587,hrh1_human,194,T,A,Histamine,ChEMBL Compound ID,CHEMBL90,Binding - unknown pharmacological activity,...,NaN,NaN,[3H]-mepyramine (radioligand),126.0,0.0,NaN,0.0,NaN,P35367,0.450196
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12016,438,10.1006/bbrc.2001.5027,aa3r_human,107,D,R,I-AB-MECA,PubChem CID,44208896,Full agonist,...,NaN,NaN,NaN,0.0,0.0,=,0.0,NaN,P0DMS8,0.757770
12017,233,10.1021/jm050968b,aa3r_human,94,T,E,CID85116573,PubChem CID,85116573,Full agonist,...,NaN,NaN,I-AB-MECA,0.0,0.0,=,0.0,NaN,P0DMS8,0.546532
12018,271,10.1021/jm050968b,aa3r_human,94,T,E,CHEMBL377071,ChEMBL Compound ID,377071,Full agonist,...,NaN,NaN,I-AB-MECA,0.0,0.0,=,0.0,NaN,P0DMS8,0.546532
12019,279,10.1021/jm050968b,aa3r_human,94,T,E,CID75022483,PubChem CID,75022483,Full agonist,...,NaN,NaN,I-AB-MECA,0.0,0.0,=,0.0,NaN,P0DMS8,0.546532


In [35]:
# VESPAl into two classifications: >=0.5(positive) <0.5(negative)
gpcr_vespa['VESPAl_group'] = gpcr_vespa['VESPAl'].apply(lambda x: 1 if x >= 0.5 else 0)
gpcr_vespa['foldchange_group'] = gpcr_vespa['exp_fold_change'].apply(lambda x: 1 if x <= -5 else (1 if x >= 5 else 0))


In [36]:
true_labels = gpcr_vespa['foldchange_group']
predicted_labels = gpcr_vespa['VESPAl_group']

In [37]:
# Calculate confusion matrix
tn, fp, fn, tp = confusion_matrix(true_labels, predicted_labels).ravel()

# Calculate accuracy
accuracy = accuracy_score(true_labels, predicted_labels)

# Calculate specificity and sensitivity
specificity = tn / (tn + fp)
sensitivity = tp / (tp + fn)


# Calculate precision, recall and F1-score
precision = precision_score(true_labels, predicted_labels)
recall = recall_score(true_labels, predicted_labels)
f1 = f1_score(true_labels, predicted_labels)

# calculate FPR and TPR
fpr, tpr, thresholds = roc_curve(true_labels, predicted_labels)

# calculate AUCROC
auc_roc = auc(fpr, tpr)



print("Specificity:", specificity)
print("Sensitivity:", sensitivity)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("F1-score:", f1)
print("AUCROC score: {:.3f}".format(auc_roc))

Specificity: 0.6598351001177857
Sensitivity: 0.4868309260832625
Accuracy: 0.6090175526162549
Precision: 0.37312784892554807
F1-score: 0.422462521504055
AUCROC score: 0.573


In [24]:
# Generate classification report
report = classification_report(true_labels, predicted_labels, labels=[0, 1], digits=4,zero_division="warn")

# Print report
print(report)


              precision    recall  f1-score   support

           0     0.8501    0.6462    0.7343      9754
           1     0.2509    0.5099    0.3363      2267

    accuracy                         0.6205     12021
   macro avg     0.5505    0.5781    0.5353     12021
weighted avg     0.7371    0.6205    0.6592     12021

